Fetching synonyms from CZI and SoftwareKG dataset 

Considering that CZI has synonyms fetched from most used software websites (PyPi, CRAN, Bioconductor and SciCrunch) this notebook will for every software in the benchmark query the datasets and try to retrieve synonyms. 

First we save all unique software names (lowered) into a dictionary, the using a function get_synonyms, we find all synonyms that have software_mention (lowered) the same as the key in the dictionary. Dictionary is used in order to speed up the process.

CZI dataset was downloaded from https://datadryad.org/dataset/doi:10.5061/dryad.6wwpzgn2c#methods (disambiguated file).

In [44]:
import pandas as pd 

df = pd.read_csv("CZI/synonyms_matrix.csv")
benchmark_df = pd.read_excel("Benchmark.xlsx")

In [45]:
print(df["software_mention"].isnull().sum())
df = df.dropna(subset=["software_mention"])
print(df["software_mention"].isnull().sum())
df['software_mention'] = df['software_mention'].astype(str)  # Convert all values to strings
benchmark_df['name']=benchmark_df['name'].astype(str)


2
0


In [46]:
benchmark_dict = {name.lower(): [] for name in benchmark_df["name"].unique()}

In [47]:
def get_synonyms(df, dictionary):
    for key in dictionary.keys():
        # Find matching rows in synonyms_df where the software mention matches the dictionary key
        matches = df[df["software_mention"].str.lower() == key]["synonym"].tolist()
        # Store synonyms as a list
        benchmark_dict[key] = ", ".join(matches)

    

In [ ]:
# Add synonyms column
get_synonyms(df, benchmark_dict)
print(benchmark_dict)
benchmark_df["synonyms"] = benchmark_df["name"].str.lower().map(benchmark_dict)

# Save the enriched benchmark
benchmark_df.to_csv("benchmark_with_synonyms.csv", index=False)

{'sklearn': 'sklearn, sklearn, Sklearn API, Scikit-learn, scikits.learn, sklearn, sklearn python package, Python sklearn package, sklearn Python library, Python package sklearn, sklearn Python, sklearn Python package, Python sklearn library, Python sklearn, sklearn.tree, sklearn.svm, sklearn-fuse, sklear, sklearn0, sklearn.utils, sklearn_extra, sklearn‐rvm, sklearn”, sklearning, sklearn.hmm, sklearn', 'sklearn python package': 'sklearn package, sklearn Python, learn Python package37, Nilearn Python package, hmmlearn Python package, sklearn python package, sklearn package, sklearn Python package', 'python package sklearn': 'Python sklearn package, Python package scikit-learn, Python packages, Python package sci-kit learn, Python sklearn, Python package scikit learn, Python package “scikit-learn”, Python package Scikit-learn, Python package scikit‐learn, Python package scikits.learn, Python package scipy, Python package Holes, Python package seaborn', 'python sklearn library': 'Python ‘s